In [1]:
# %pip install -U lsdb hats-import

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.6 MB/s eta 0:00:00
  Attempting uninstall: dask
    Found existing installation: dask 2025.4.1
    Uninstalling dask-2025.4.1:
      Successfully uninstalled dask-2025.4.1
  Attempting uninstall: distributed
    Found existing installation: distributed 2025.4.1
    Uninstalling distributed-2025.4.1:
      Successfully uninstalled distributed-2025.4.1
  Attempting uninstall: hats-import
    Found existing installation: hats-import 0.5.1
    Uninstalling hats-import-0.5.1:
      Successfully uninstalled hats-import-0.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-expr 1.1.16 requires dask==2024.10.0, but you have dask 2025.3.0 which is incompatible.
nested-dask 0.3.4 requires nested-pandas<0.4.0,>=0.3.1, but you have neste

In [2]:
from functools import cache
from pathlib import Path

import lsdb
import pandas as pd
from dask.distributed import Client
from hats_import.pipeline import pipeline_with_client
from hats_import.catalog.arguments import ImportArguments

In [3]:
# Cache the function output to not hurt the SNAD server too much
@cache
def get_tns_df():
    df = pd.read_json("https://tns.snad.space/api/v1/all")
    df = df.drop("coord", axis=1)
    df = df.rename(columns={"declination": "dec"})
    # Fast hack to make discovery date filtering simplier
    df['discoveryyear'] = df['discoverydate'].str[:4].astype(int)
    return df

In [4]:
# Create lsdb.Catalog object
tns = lsdb.from_dataframe(
    get_tns_df()
).query(
    '2018 <= discoveryyear <= 2023',
)

In [5]:
# Load ZTF alerts catalog metadata
# "Nest" light curves, references and non-detections
raw_catalog = lsdb.read_hats(
    'https://data.lsdb.io/hats/alerce/',
)
display(raw_catalog)

# Pack all list-columns into single column
catalog_with_lc = raw_catalog.nest_lists(
    base_columns=[col for col in raw_catalog.columns if not col.startswith("lc_")],
    name="lc",
)

# Pack non-detections
catalog_with_nondet = catalog_with_lc.nest_lists(
    base_columns=[col for col in catalog_with_lc.columns if not col.startswith("nondet_")],
    name="nondet",
)

# Pack ZTF references
alerce = catalog_with_nondet.nest_lists(
    base_columns=[col for col in catalog_with_nondet.columns if not col.startswith("ref_")],
    name="ref",
)

alerce

,oid,mean_ra,mean_dec,lc_ra,lc_dec,lc_candid,lc_mjd,lc_fid,lc_pid,lc_diffmaglim,lc_isdiffpos,lc_nid,lc_magpsf,lc_sigmapsf,lc_magap,lc_sigmagap,lc_distnr,lc_rb,lc_rbversion,lc_drb,lc_drbversion,lc_magapbig,lc_sigmagapbig,lc_rfid,lc_magpsf_corr,lc_sigmapsf_corr,lc_sigmapsf_corr_ext,lc_corrected,lc_dubious,lc_parent_candid,lc_has_stamp,lc_step_id_corr,nondet_mjd,nondet_fid,nondet_diffmaglim,ref_rfid,ref_candid,ref_fid,ref_rcid,ref_field,ref_magnr,ref_sigmagnr,ref_chinr,ref_sharpnr,ref_ranr,ref_decnr,ref_mjdstartref,ref_mjdendref,ref_nframesref,Norder,Dir,Npix
npartitions=113,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Order: 1, Pixel: 0",string[pyarrow],double[pyarrow],double[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: int64>[pyarrow],list<element: double>[pyarrow],list<element: int64>[pyarrow],list<element: int64>[pyarrow],list<element: double>[pyarrow],list<element: int64>[pyarrow],list<element: int64>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: string>[pyarrow],list<element: double>[pyarrow],list<element: string>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: bool>[pyarrow],list<element: bool>[pyarrow],list<element: double>[pyarrow],list<element: bool>[pyarrow],list<element: string>[pyarrow],list<element: double>[pyarrow],list<element: int64>[pyarrow],list<element: double>[pyarrow],list<element: int64>[pyarrow],list<element: int64>[pyarrow],list<element: int64>[pyarrow],list<element: int64>[pyarrow],list<element: int64>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: double>[pyarrow],list<element: int64>[pyarrow],int8[pyarrow],int64[pyarrow],int64[pyarrow]
"Order: 2, Pixel: 4",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 0, Pixel: 10",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 0, Pixel: 11",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


,oid,mean_ra,mean_dec,Norder,Dir,Npix,lc,nondet,ref
npartitions=113,,,,,,,,,
"Order: 1, Pixel: 0",string[pyarrow],double[pyarrow],double[pyarrow],int8[pyarrow],int64[pyarrow],int64[pyarrow],"nested<lc_ra: [double], lc_dec: [double], lc_c...","nested<nondet_mjd: [double], nondet_fid: [int6...","nested<ref_rfid: [int64], ref_candid: [int64],..."
"Order: 2, Pixel: 4",...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
"Order: 0, Pixel: 10",...,...,...,...,...,...,...,...,...
"Order: 0, Pixel: 11",...,...,...,...,...,...,...,...,...


In [6]:
# Create a cross-matched catalog object
xmatch = tns.crossmatch(
    alerce,
    radius_arcsec=3.0,
    n_neighbors=1,
    suffixes=['_tns', ''],
)

/astro/users/kmalanch/.virtualenvs/default/lib/python3.10/site-packages/lsdb/dask/crossmatch_catalog_data.py:147: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(


In [7]:
%%time

# Actually do the job and write results to disk

name = 'tns_alerce'
with Client(n_workers=16, threads_per_worker=1, memory_limit='8GB') as client:
    display(client)
    xmatch.to_hats(
        Path(".") / name,  # path
        catalog_name=name,  # HATS catalog name
    )

/astro/users/kmalanch/.virtualenvs/default/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34921 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:34921/status,
Dashboard: http://127.0.0.1:34921/status,Workers: 16
Total threads: 16,Total memory: 119.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42979,Workers: 16
Dashboard: http://127.0.0.1:34921/status,Total threads: 16
Started: Just now,Total memory: 119.21 GiB
Comm: tcp://127.0.0.1:40005,Total threads: 1
Dashboard: http://127.0.0.1:36512/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:40361,


/astro/users/kmalanch/.virtualenvs/default/lib/python3.10/site-packages/distributed/client.py:3383: UserWarning: Sending large graph of size 92.26 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


CPU times: user 30.8 s, sys: 12.9 s, total: 43.7 s
Wall time: 1min 26s


In [8]:
%%time

# DOESN'T WORK RIGHT NOW, I KEEP IT FOR A BUG REPORT

# Resample the catalog for larger parquet files,
# up to 1000 objects per file

new_name = f'{name}_1k'
args = ImportArguments.reimport_from_hats(
    Path(".") / name,
    Path("."),
    output_artifact_name=new_name,
    pixel_threshold=1_000,
)

with Client(n_workers=16, threads_per_worker=1, memory_limit='8GB') as client:
    display(client)
    pipeline_with_client(args, client)

Validating catalog at path tns_alerce ... 
Found 113 partitions.
Approximate coverage is 97.92 % of the sky.


/astro/users/kmalanch/.virtualenvs/default/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40948 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:40948/status,
Dashboard: http://127.0.0.1:40948/status,Workers: 16
Total threads: 16,Total memory: 119.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45411,Workers: 16
Dashboard: http://127.0.0.1:40948/status,Total threads: 16
Started: Just now,Total memory: 119.21 GiB
Comm: tcp://127.0.0.1:35430,Total threads: 1
Dashboard: http://127.0.0.1:39049/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:37367,


Planning  :   0%|          | 0/4 [00:00<?, ?it/s]

Mapping   :   0%|          | 0/113 [00:00<?, ?it/s]

Binning   :   0%|          | 0/2 [00:00<?, ?it/s]

TypeError: cannot unpack non-iterable numpy.int32 object